In [2]:
import pandas as pd
import numpy as np
import pyCloudy as pc
import os
pc.log_.level = 3;

warng pyCloudy config: pyCloudy works better with PyNeb


In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import display

##### This will be the style for the button font

In [4]:
%%html
<style>.myclass { color:white ; font-size:150%}

 ## Create Input Script

In [35]:
# Define path to executable
dir_ = './'
make='Model5' #will also be punch prefix
pathtorun='{0}{1}'.format(dir_, make) #path to input script
pathtoexec='/Users/danielpiacitelli/nublado/c17.02/source/cloudy.exe' #path to Cloudy executable 

##### Temperature command 

In [12]:
Topt='CONSTANT TEMPERATURE' #can also be SET TEMP

##### Choose of which lines emisitivity is needed 

In [13]:
emis = ['H  1  4861.33A',
        'H  1  6562.81A', 
        'O 6  1031.00A', 
        'O  6 1037.62A']

In [14]:
opti = ['CMB redshift 0.2000000',
         'no molecules',
         'Print column densities on'] #mostly miscellaneous manipulations

In [15]:
def createinput(Temperature, HDensity, Metallicity):
    T=Temperature #K Linear
    H=HDensity #Log /cm^3
    M=Metallicity #Log 
    distance= 300000.0 # Linear kpc and mostly meaningless for CGM purposes
    manip= ['{0} {1}'.format('metals', str(M))] + ['{0} {1} {2}'.format(Topt, '1e0'+str(T), 'K linear')] + opti
    stop= ['proton column density 15.500000','temperature = 10000K [linear]'] #stopping criteria
    run = pc.CloudyInput(pathtorun);
    run.set_cste_density(H, ff = None );run.set_distance(dist=distance, unit='kpc', linear=True);
    run.set_abund(nograins = True);
    run.set_iterate() # (0) = no iteration, () = 1 iteration, (N) = N iterations
    run.set_emis_tab(emis); #insert list of desired lines
    run.set_other(manip);
    run.set_stop(stop); #will insert our stopping criteria
    run.print_input(to_file = True, verbose = False)
    return run
print("Temperature is in Kelvin and Linear, HDensity is in /cm^3 and Log, Metallicity is Log")
T=widgets.IntSlider(value=5,min=0,step=1,max=8); H=widgets.FloatSlider(value=-2.5,min=-6,max=1,step=0.25); M=widgets.IntSlider(value=-1,min=-5,max=1,step=1);
ui = widgets.HBox([T, H, M])
T.style.handle_color = '#738678';H.style.handle_color = '#738678';M.style.handle_color = '#738678';
interact(createinput,Temperature=T,HDensity=H,Metallicity=M);

Temperature is in Kelvin and Linear, HDensity is in /cm^3 and Log, Metallicity is Log


interactive(children=(IntSlider(value=5, description='Temperature', max=8, style=SliderStyle(handle_color='#73…

In [10]:
b = Button(description='Run Cloudy with last run file',layout=Layout(width='50%', height='80px'))
b.style.button_color = '#738678';b.add_class('myclass')
output = widgets.Output()
def runCloudy(b):
    run = pc.CloudyInput(pathtorun);
    pc.config.cloudy_exe = pathtoexec;
    pc.log_.timer('Starting Cloudy', quiet = True)
    run.run_cloudy()
    with output:
        pc.log_.message('Ran {0}'.format(make))
        pc.log_.timer('Cloudy ended after seconds:')   
display(b, output)
b.on_click(runCloudy)

Button(description='Run Cloudy with last run file', layout=Layout(height='80px', width='50%'), style=ButtonSty…

Output()

## Pandas Tables

##### Returns ovr table with Temperature, Htot, hydrogen density, e density, and OVI density

In [39]:
vari='.ovr'
table=pd.read_csv('{0}{1}'.format(make, vari),delimiter='	')
ovr=table.loc[:,['#depth','Te','Htot','hden','eden','O5']]
#Temp=np.log10(ovr.loc[1,"Te"])
ovr

,#depth,Te,Htot,hden,eden,O5


##### Returns emis table with Emissivity of H 1 4861.33A, H 1 6562.81A, O 6 1031.00A, O 6 1037.62A

In [38]:
vari='.emis'
emis=pd.read_csv('{0}{1}'.format(make, vari),delimiter='	')
emis.columns=['#depth','H1-4861.33A', 'H1-6562.81A', 'O6-1031.91A','O6-1037.62A']

#isolates the 1031 and 1037 lines as independent tables
tenthirtyone= emis.loc[:,'O6-1031.91A'] 
tenthirtyseven= emis.loc[:,'O6-1037.62A']

#tenthirtyone
#tenthirtyseven
emis

,#depth,H1-4861.33A,H1-6562.81A,O6-1031.91A,O6-1037.62A
0,2.500000e+16,0.0,0.0,0.0,0.0


##### Returns eleH table with H	H+	H2 probabilites for each zone

In [23]:
#returns eleH table with H	H+	H2 probabilites for each zone

vari='.ele_H'
eleH=pd.read_csv('{0}{1}'.format(make, vari),delimiter='	')
#print(eleH)
#returns eleO table with O	O+	O+2	O+3	O+4	O+5	O+6	O+7	O+8	O[1]	O[2]	O[3] probabilites for each zone
vari='.ele_O'
eleO=pd.read_csv('{0}{1}'.format(make,vari),delimiter='	')
#eleO

##### Returns table OVI_prob with OVI probabilites for each zone

In [24]:
OVI_prob = eleO.loc[:,['#depth', 'O+5']]
#OVI_prob

## Calculations

##### Returns single float of OVI fraction, electron density, emissivity of OVI1031/OVI1037, and total emissivity of OVI

In [25]:
O6frac=np.average(OVI_prob.loc[:,'O+5'].values)
#O6total=np.average(ovr.loc[:,'eden'].values)
eden=np.average(ovr.loc[:,'eden'].values)
emis1031=np.average(tenthirtyone.values)
emis1037=np.average(tenthirtyseven.values)
hden=np.average(ovr.loc[:,'hden'].values)

print('The OVI fraction is',O6frac)
#print('OVI total is',O6total)
print('The electron density is',eden)
print('The emissivity of OVI1031 is',emis1031)
print('The emissivity of OVI1037 is',emis1037)
print('The total emissivity of OVI is',emis1037+emis1037)

The OVI fraction is 0.0
The electron density is 2.0927e-17
The emissivity of OVI1031 is 0.0
The emissivity of OVI1037 is 0.0
The total emissivity of OVI is 0.0


##### Gives number of oxygnVI and Column Densities

In [26]:
Noxy_Sol=10**-3.309
Zsol=10**-1
nOVI=O6frac*Zsol*Noxy_Sol
print('nOVI is',nOVI)
length=ovr.loc[:,"#depth"].max() -  ovr.loc[:,"#depth"].min()
N_OVI=nOVI*length
print('Column density of OVI is',N_OVI)

nOVI is 0.0
Column density of OVI is 0.0


## This will give you the list of classes for th cloudy output

In [27]:
#Mod = pc.CloudyModel(pathtorun);
# dir(Mod) #remove # at beginning to print list

In [11]:
Mod.print_lines()
Mod.print_stats()

H__1_486133A 1.931812e+44
H__1_656281A 5.251109e+44
O__6_103100A 2.411241e+40
O__6_103762A 1.199328e+40
 Name of the model: ./ASTR302
 R_in (cut) = 1.000e+30 (1.000e+30), R_out (cut) = 1.000e+30 (1.000e+30)
 H+ mass = 3.34e+19, H mass = 3.34e+19 N zones: 33
 <H+/H> = 1.00, <He++/He> = 1.00, <He+/He> = 0.00
 <O+++/O> = 0.00, <O++/O> = 0.00, <O+/O> = 0.00
 <N+++/N> = 0.00, <N++/N> = 0.00, <N+/N> = 0.00
 T(O+++) = 100000000, T(O++) = 100000000, T(O+) = 100000000
 <ne> = 0,  <nH> = 0, T0 = 100000000, t2=0.0000
 <log U> = -2.11


## Save Simulation data for future use

In [170]:
# run this once 
location=0

In [28]:
# run this once 
SimuDatKeyset=['H Density Log /cm^3','e Density Log /cm^3','OVI Probability','OVI 1031+1037 Emis']
SimnDat=[hden, eden,O6frac,emis1031+emis1037]
matrix={SimuDatKeyset[0]:[SimnDat[0]],SimuDatKeyset[1]:[SimnDat[1]],SimuDatKeyset[2]:[SimnDat[2]],SimuDatKeyset[3]:[SimnDat[3]]}
PastSimulationData=pd.DataFrame(data=matrix,index={f"Temp:1e{Temp:1.1f} K , hden: {np.log10(hden):1.1f}"}).transpose()
PastSimulationData

,"Temp:1e3.0 K , hden: -2.5"
H Density Log /cm^3,3.162300e-03
e Density Log /cm^3,2.092700e-17
OVI Probability,0.000000e+00
OVI 1031+1037 Emis,0.000000e+00


In [198]:
SimnDat=[hden, eden,O6frac,emis1031+emis1037]

##### This will add new sim data to table

In [221]:
location=location+1
PastSimulationData.insert(location,f"Temp:1e{Temp:1.1f} K , hden: {np.log10(hden):1.1f}",SimnDat)

In [222]:
PastSimulationData

,"Temp:1e7.0 K , hden: -2.5","Temp:1e7.0 K , hden: -2.0","Temp:1e8.0 K , hden: -2.5"
H Density Log /cm^3,3.162300e-03,1.000000e-02,1.000000e-02
e Density Log /cm^3,3.797497e-03,1.200900e-02,1.200900e-02
OVI Probability,3.160000e-07,3.170000e-07,3.170000e-07
OVI 1031+1037 Emis,5.741494e-35,5.555100e-34,5.555100e-34


### Save PastSimulationData as csv file

In [ ]:
PastSimulationData.to_csv('./CloudyDataTables/SimulationData.csv',index=True)